In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
import joblib
from sklearn.model_selection import train_test_split


In [4]:
df_ocs1 = pd.read_csv("datatrain/3M_OCS.csv", delimiter=';') 
catalog = pd.read_csv("datatrain/3M_CATALOG.csv", delimiter=';') 

In [5]:
catalog['3M SKU:']

0       UU010885166
1       UU010903167
2       UU010903175
3       UU010903191
4       UU010899217
           ...     
2485    XH000019444
2486    XH000019535
2487    SH000019530
2488    SH000019639
2489    70202028182
Name: 3M SKU:, Length: 2490, dtype: object

In [7]:
df_ocs1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10228 entries, 0 to 10227
Data columns (total 2 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Codigo                 10228 non-null  object
 1   Descripcion Comprador  10228 non-null  object
dtypes: object(2)
memory usage: 159.9+ KB


In [34]:
import pandas as pd

#UPSAMPLING PARA DAR IMPORTANCIA A CLASES MINORITARIAS, SIN SOREAJUSTAR
umbral_frecuencia = 40


frecuencia_clases = df_ocs1['Codigo'].value_counts()
clases_frecuencia_baja = frecuencia_clases[frecuencia_clases < umbral_frecuencia].index
df_upsampled = pd.DataFrame(columns=df_ocs1.columns)

# Iterar sobre las clases con frecuencia baja
for clase in clases_frecuencia_baja:
    filas_clase = df_ocs1[df_ocs1['Codigo'] == clase]
    registros_faltantes = umbral_frecuencia - len(filas_clase)
    nuevos_registros = pd.concat([filas_clase] * (registros_faltantes // len(filas_clase) + 1), ignore_index=True)
    nuevos_registros = nuevos_registros.head(registros_faltantes)
    df_upsampled = pd.concat([df_upsampled, nuevos_registros])

df_upsampled['UP_SAMPLING'] = True

df_enteral_upsampled = pd.concat([df_ocs1, df_upsampled])

df_enteral_upsampled.reset_index(drop=True, inplace=True)

In [35]:
import pandas as pd

# Definir el umbral de frecuencia
umbral_frecuencia = 40

# Calcular la frecuencia de las clases
frecuencia_clases = df_ocs1['Codigo'].value_counts()

# Identificar clases con frecuencia baja
clases_frecuencia_baja = frecuencia_clases[frecuencia_clases < umbral_frecuencia].index

# Inicializar un DataFrame para el upsampling
df_upsampled = pd.DataFrame(columns=df_ocs1.columns)

# Iterar sobre todas las clases
for clase in frecuencia_clases.index:
    filas_clase = df_ocs1[df_ocs1['Codigo'] == clase]
    num_registros_clase = len(filas_clase)
    
    # Nivelar la cantidad de registros según el umbral
    if num_registros_clase < umbral_frecuencia:
        nuevos_registros = pd.concat([filas_clase] * ((umbral_frecuencia // num_registros_clase) + 1), ignore_index=True)
        nuevos_registros = nuevos_registros.head(umbral_frecuencia)
    elif num_registros_clase > umbral_frecuencia:
        nuevos_registros = filas_clase.sample(n=umbral_frecuencia, replace=False)
    else:
        nuevos_registros = filas_clase
    
    df_upsampled = pd.concat([df_upsampled, nuevos_registros])

df_upsampled['UP_SAMPLING'] = True

# Combinar DataFrames
#df_enteral_upsampled = pd.concat([df_ocs1, df_upsampled])

# Reiniciar índices
#df_enteral_upsampled.reset_index(drop=True, inplace=True)


In [36]:
df_upsampled['Codigo'].value_counts()

70201059907    40
90206891450    40
70202091529    40
70202067479    40
70202119593    40
               ..
70202091040    40
70202073451    40
70201059840    40
70201144386    40
70202072990    40
Name: Codigo, Length: 454, dtype: int64

In [38]:
df_enteral_upsampled.fillna('', inplace=True)
#df = df_enteral_upsampled.copy()
df = df_upsampled #menos valores para generar pkl menor a 2gb

nltk.download('stopwords')
stop_words = set(stopwords.words('spanish'))
df['Descriptions']= df['Descripcion Comprador'].apply(lambda x: ' '.join([word for word in x.split() if word.lower() not in stop_words]))


# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(df['Descriptions'], df['Codigo'], test_size=0.2, stratify=df['Codigo'], random_state=42)

# Definir el vectorizador TF-IDF
tfidf_vectorizer = TfidfVectorizer()

# Ajustar el vectorizador TF-IDF solo al conjunto de entrenamiento
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)

# Guardar el vectorizador TF-IDF en un archivo pkl
joblib.dump(tfidf_vectorizer, 'model/tfidf_vectorizer-1.1.0.pkl')

# Definir el modelo RandomForestClassifier
rf_classifier = RandomForestClassifier()

# Ajustar el modelo RandomForestClassifier solo al conjunto de entrenamiento
rf_classifier.fit(X_train_tfidf, y_train)

# Guardar el modelo RandomForestClassifier en un archivo pkl
joblib.dump(rf_classifier, 'model/random_forest_classifier-1.1.0.pkl')



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ngonzalez\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['model/random_forest_classifier-1.1.0.pkl']

In [39]:
from sklearn.metrics import classification_report

# Realizar predicciones en el conjunto de prueba
X_test_tfidf = tfidf_vectorizer.transform(X_test)
y_pred = rf_classifier.predict(X_test_tfidf)

# Generar el informe de clasificación
report = classification_report(y_test, y_pred)

# Imprimir el informe de clasificación
print(report)


              precision    recall  f1-score   support

 70000800188       0.89      1.00      0.94         8
 70000807167       1.00      1.00      1.00         8
 70000807175       0.80      1.00      0.89         8
 70000807209       1.00      1.00      1.00         8
 70000807217       1.00      1.00      1.00         8
 70000821218       1.00      1.00      1.00         8
 70000821226       0.89      1.00      0.94         8
 70000836851       1.00      0.75      0.86         8
 70000836869       1.00      1.00      1.00         8
 70000836877       1.00      1.00      1.00         8
 70000836885       1.00      1.00      1.00         8
 70000836901       0.80      1.00      0.89         8
 70000846405       0.89      1.00      0.94         8
 70000846413       1.00      0.88      0.93         8
 70000846421       0.53      1.00      0.70         8
 70000846439       0.67      0.75      0.71         8
 70000846447       1.00      0.25      0.40         8
 70000846454       0.62    

In [28]:
joblib.dump(pipeline, 'model/trained_3m-0.1.0.joblib')

['model/trained_3m-0.1.0.joblib']

In [27]:


# Guardar la variable stop_words en un archivo pkl
joblib.dump(stop_words, 'stopwords.pkl')


['stopwords.pkl']